In [1]:
import cv2
import numpy as np
import xml.etree.ElementTree as ET
from collections import defaultdict



In [2]:
from __future__ import absolute_import, division, print_function

import os
import numpy as np
from collections import Counter


def load_velodyne_points(filename):
    """Load 3D point cloud from KITTI file format
    (adapted from https://github.com/hunse/kitti)
    """
    points = np.fromfile(filename, dtype=np.float32).reshape(-1, 4)
    points[:, 3] = 1.0  # homogeneous
    return points


def read_calib_file(path):
    """Read KITTI calibration file
    (from https://github.com/hunse/kitti)
    """
    float_chars = set("0123456789.e+- ")
    data = {}
    with open(path, 'r') as f:
        for line in f.readlines():
            key, value = line.split(':', 1)
            value = value.strip()
            data[key] = value
            if float_chars.issuperset(value):
                # try to cast to float array
                try:
                    data[key] = np.array(list(map(float, value.split(' '))))
                except ValueError:
                    # casting error: data[key] already eq. value, so pass
                    pass

    return data


def sub2ind(matrixSize, rowSub, colSub):
    """Convert row, col matrix subscripts to linear indices
    """
    m, n = matrixSize
    return rowSub * (n-1) + colSub - 1


def generate_depth_map(calib_dir, velo_filename, cam=2, vel_depth=False):
    """Generate a depth map from velodyne data
    """
    # load calibration files
    cam2cam = read_calib_file(os.path.join(calib_dir, 'calib_cam_to_cam.txt'))
    velo2cam = read_calib_file(os.path.join(calib_dir, 'calib_velo_to_cam.txt'))
    velo2cam = np.hstack((velo2cam['R'].reshape(3, 3), velo2cam['T'][..., np.newaxis]))
    velo2cam = np.vstack((velo2cam, np.array([0, 0, 0, 1.0])))

    # get image shape
    im_shape = cam2cam["S_rect_02"][::-1].astype(np.int32)

    # compute projection matrix velodyne->image plane
    R_cam2rect = np.eye(4)
    R_cam2rect[:3, :3] = cam2cam['R_rect_00'].reshape(3, 3)
    P_rect = cam2cam['P_rect_0'+str(cam)].reshape(3, 4)
    P_velo2im = np.dot(np.dot(P_rect, R_cam2rect), velo2cam)

    # load velodyne points and remove all behind image plane (approximation)
    # each row of the velodyne data is forward, left, up, reflectance
    velo = load_velodyne_points(velo_filename)
    velo = velo[velo[:, 0] >= 0, :]

    # project the points to the camera
    velo_pts_im = np.dot(P_velo2im, velo.T).T
    velo_pts_im[:, :2] = velo_pts_im[:, :2] / velo_pts_im[:, 2][..., np.newaxis]

    if vel_depth:
        velo_pts_im[:, 2] = velo[:, 0]

    # check if in bounds
    # use minus 1 to get the exact same value as KITTI matlab code
    velo_pts_im[:, 0] = np.round(velo_pts_im[:, 0]) - 1
    velo_pts_im[:, 1] = np.round(velo_pts_im[:, 1]) - 1
    val_inds = (velo_pts_im[:, 0] >= 0) & (velo_pts_im[:, 1] >= 0)
    val_inds = val_inds & (velo_pts_im[:, 0] < im_shape[1]) & (velo_pts_im[:, 1] < im_shape[0])
    velo_pts_im = velo_pts_im[val_inds, :]

    # project to image
    depth = np.zeros((im_shape[:2]))
    depth[velo_pts_im[:, 1].astype(np.int), velo_pts_im[:, 0].astype(np.int)] = velo_pts_im[:, 2]

    # find the duplicate points and choose the closest depth
    inds = sub2ind(depth.shape, velo_pts_im[:, 1], velo_pts_im[:, 0])
    dupe_inds = [item for item, count in Counter(inds).items() if count > 1]
    for dd in dupe_inds:
        pts = np.where(inds == dd)[0]
        x_loc = int(velo_pts_im[pts[0], 0])
        y_loc = int(velo_pts_im[pts[0], 1])
        depth[y_loc, x_loc] = velo_pts_im[pts, 2].min()
    depth[depth < 0] = 0

    return depth


In [3]:
# just like before we specify an enum flag, but this time it is 
# FILE_STORAGE_READ
mappoint = cv2.FileStorage("6.xml", cv2.FILE_STORAGE_READ)
keypoint = cv2.FileStorage("../Results/6.xml", cv2.FILE_STORAGE_READ)
sysmappoint = cv2.FileStorage("../Results_map_system/6.xml", cv2.FILE_STORAGE_READ)

# for some reason __getattr__ doesn't work for FileStorage object in python
# however in the C++ documentation, getNode, which is also available, 
# does the same thing
#note we also have to specify the type to retrieve other wise we only get a 
# FileNode object back instead of a matrix

depth_map = mappoint.getNode("Depth").mat()
depth_key = keypoint.getNode("Depth").mat()
depth_map_sys = sysmappoint.getNode("Depth").mat()

mappoint.release()
keypoint.release()
sysmappoint.release()

depth_map = np.asarray(depth_map)
depth_key = np.asarray(depth_key)
depth_map_sys = np.asarray(depth_map_sys)

print(np.max(depth_key))
print(np.max(depth_map_sys))
print(np.max(depth_map))


9321.936
9322.115
3657.1462


In [4]:
# print(depth.shape, np.max(depth), np.min(depth))
N = 0
test_depth = depth_key
for i in range(375):
    for j in range(1242):
        if test_depth[i][j] != 0.0 and test_depth[i][j] <= 70:
            N+=1
            print((i,j), test_depth[i][j])
print("number of anchors", N)


(21, 294) 29.66002
(24, 1023) 15.958005
(26, 297) 34.402573
(26, 991) 16.494413
(26, 1088) 13.5643215
(27, 1085) 13.54389
(29, 249) 29.71771
(30, 66) 18.592777
(30, 67) 18.215313
(30, 1052) 13.770462
(31, 796) 34.220463
(34, 748) 38.559937
(34, 769) 41.50423
(37, 391) 61.0035
(38, 748) 39.54229
(38, 1000) 16.116573
(38, 1105) 12.118705
(39, 94) 18.401524
(39, 748) 42.171738
(39, 1002) 16.002213
(39, 1103) 12.524144
(40, 95) 18.517591
(40, 760) 46.95998
(40, 848) 36.86378
(41, 266) 29.521402
(41, 684) 21.827852
(41, 1036) 15.136981
(42, 684) 23.448854
(43, 724) 21.739573
(43, 1173) 10.491221
(44, 421) 44.870373
(44, 468) 54.0094
(44, 808) 35.381996
(44, 1178) 10.789204
(45, 468) 55.04746
(45, 808) 34.584938
(46, 420) 45.89663
(46, 783) 36.361458
(46, 803) 37.187702
(47, 169) 28.143532
(47, 192) 29.618437
(47, 421) 46.063477
(47, 467) 52.23877
(47, 922) 21.584644
(47, 923) 20.752794
(48, 170) 28.883558
(48, 171) 29.61702
(48, 924) 19.958431
(49, 432) 39.08845
(49, 434) 46.290367
(49, 923

(312, 927) 8.39543
(312, 1036) 7.9186583
(313, 791) 7.9914875
(313, 926) 8.485497
(313, 928) 8.317104
(313, 946) 8.365207
(314, 906) 8.311453
(316, 820) 8.02799
(316, 851) 7.707521
(317, 991) 7.6918225
(318, 851) 7.861079
(319, 77) 7.037519
(319, 240) 7.695149
(322, 331) 8.636242
(322, 1121) 7.0192194
(322, 1122) 7.123983
(323, 821) 7.6195526
(323, 1119) 7.277035
(323, 1121) 7.1166353
(323, 1129) 7.061287
(325, 950) 7.5972395
(326, 950) 7.6575766
(326, 964) 7.5846405
(328, 783) 7.1185265
(329, 828) 7.395638
(330, 1171) 6.964346
(331, 218) 7.3614497
(331, 219) 7.294086
(331, 491) 7.2587585
(331, 492) 7.283276
(331, 493) 7.277081
(331, 494) 7.251864
(331, 588) 7.4317064
(331, 1107) 6.8956265
(332, 215) 7.013785
(332, 495) 7.294461
(332, 1094) 7.0035458
(332, 1108) 6.871027
(333, 580) 1.5899096
(333, 860) 7.0523596
(333, 874) 6.9734573
(334, 874) 7.089154
(335, 685) 7.307005
(336, 687) 7.1332526
(336, 799) 6.881074
(338, 200) 6.988417
(338, 332) 7.105368
(338, 333) 7.0151157
(338, 639) 7.

In [10]:

# print(depth.shape, np.max(depth), np.min(depth))
N = 0
test_depth = depth_map
print(test_depth.shape)
for i in range(375):
    for j in range(1242):
        if test_depth[i][j] != 0.0 and test_depth[i][j] <= 70 and j>1000:
            N+=1
            print((i,j), test_depth[i][j])
print("number of anchors", N)

(375, 1242)
(2, 1105) 13.052641
(3, 1103) 13.314764
(8, 1080) 13.584948
(10, 1074) 14.420137
(12, 1038) 14.459026
(20, 1114) 12.664814
(20, 1117) 12.16416
(20, 1120) 12.087404
(21, 1117) 12.176567
(22, 1184) 9.376644
(25, 1091) 13.052748
(26, 1089) 13.220329
(26, 1235) 10.651279
(28, 1170) 11.063713
(29, 1053) 14.155838
(29, 1055) 13.927652
(29, 1057) 13.288883
(29, 1158) 11.908112
(29, 1165) 11.576403
(31, 1236) 10.576998
(32, 1045) 15.10548
(32, 1232) 10.748388
(33, 1146) 11.197128
(33, 1147) 11.913982
(35, 1136) 11.997908
(36, 1137) 11.900362
(40, 1181) 10.239431
(45, 1126) 11.279463
(45, 1164) 10.92127
(46, 1019) 15.499727
(46, 1164) 10.936541
(47, 1117) 11.605757
(49, 1111) 11.983915
(52, 1106) 12.215311
(55, 1141) 12.129129
(55, 1231) 11.35795
(56, 1138) 11.436966
(56, 1142) 11.986824
(56, 1233) 10.751984
(56, 1234) 10.581953
(58, 1014) 15.927322
(59, 1127) 12.194704
(64, 1051) 14.29575
(64, 1114) 12.1093445
(73, 1078) 13.147966
(73, 1080) 13.143497
(74, 1090) 12.440245
(84, 1122

In [11]:
calib_dir = "/home/hanzhic/KITTI_Depth_Demo/calib/"
velo_filename = "/home/hanzhic/KITTI_Depth_Demo/2011_09_26/2011_09_26_drive_0001_sync/velodyne_points/data/0000000006.bin"

generate_depth_map(calib_dir, velo_filename, cam=2, vel_depth=False)
N = 0
test_depth = generate_depth_map(calib_dir, velo_filename, cam=2, vel_depth=False)
print(np.max(test_depth))
for i in range(375):
    for j in range(1242):
        if test_depth[i][j] != 0.0:
            N+=1
            print((i,j), test_depth[i][j])
print("number of anchors", N)

78.91442348618034
(121, 1232) 10.830154461345627
(121, 1236) 10.782143932902352
(121, 1241) 10.730135239556622
(122, 1214) 11.02118892528314
(122, 1226) 10.949178508000864
(122, 1230) 10.937183436592225
(123, 1204) 11.461296580365552
(123, 1207) 11.07518450396432
(123, 1210) 11.283260186047714
(124, 1193) 11.567313252775605
(124, 1196) 11.539310178061852
(124, 1200) 11.483300085818563
(127, 1235) 10.818191250871617
(127, 1239) 10.752173595019205
(128, 1213) 10.982205357187715
(128, 1223) 10.924196601559375
(128, 1227) 10.734166499408616
(129, 1201) 11.138224220674514
(129, 1205) 11.268259547388801
(129, 1209) 11.0002105733226
(129, 1216) 11.371293935972314
(129, 1220) 11.274269996246
(130, 1116) 14.990203598552826
(130, 1119) 14.946189950101083
(130, 1197) 11.512310488472034
(130, 1210) 19.150632575838227
(130, 1214) 18.882555817580283
(131, 1090) 15.121197852129352
(131, 1094) 15.076195383040956
(131, 1097) 15.021184479908188
(131, 1109) 15.086217977169307
(131, 1112) 15.0362054637433

(200, 1025) 16.53467116321713
(200, 1028) 16.409730370409083
(200, 1031) 16.273801944881885
(200, 1034) 16.262784579871656
(200, 1036) 16.2597699284728
(200, 1038) 26.731686036936818
(200, 1041) 26.62171847695686
(200, 1044) 26.53172610430321
(200, 1047) 26.419756310758398
(200, 1050) 26.318786757176984
(200, 1053) 26.200819176458683
(200, 1057) 26.11282762766948
(200, 1058) 26.03486300730336
(200, 1061) 25.935881793897664
(200, 1065) 25.85288876035825
(200, 1068) 25.764907911331612
(200, 1071) 25.661927580261374
(200, 1074) 25.553959433703906
(200, 1076) 25.49697117947645
(200, 1079) 25.397990215525027
(200, 1082) 25.318007852379104
(200, 1086) 25.207029408556544
(200, 1089) 25.13204759181621
(200, 1092) 24.947119899903655
(200, 1096) 24.836141704675217
(200, 1099) 24.709187937447677
(200, 1101) 24.575250962161974
(200, 1104) 24.411312119958207
(200, 1107) 24.228385749454162
(200, 1111) 24.06744712709839
(200, 1114) 23.8765330953962
(200, 1118) 23.705595909670656
(200, 1121) 23.548654

(280, 1015) 10.399828571051048
(280, 1018) 10.34190284037713
(280, 1020) 10.272013596689884
(280, 1023) 10.089313253169994
(280, 1029) 9.986440294722083
(280, 1033) 10.009358678868542
(280, 1036) 9.944435060869631
(280, 1039) 9.857566205937337
(280, 1041) 9.79566524380626
(280, 1044) 9.725763184192205
(280, 1047) 9.654860900396717
(280, 1050) 9.579959496889943
(280, 1054) 9.514046353967819
(280, 1057) 9.421188629033004
(280, 1060) 9.343297954835776
(280, 1062) 9.286386134823955
(280, 1065) 9.20750676502869
(280, 1081) 10.298777442630822
(280, 1084) 10.252819041517848
(280, 1086) 10.250795656031308
(280, 1089) 10.27271402229379
(280, 1092) 10.213778942456987
(280, 1096) 10.228707715864166
(280, 1181) 10.210508798589796
(280, 1184) 10.112641775396334
(280, 1186) 10.096641238410403
(280, 1190) 10.029729072514519
(281, 1069) 9.155560571891794
(281, 1072) 9.09364704497912
(281, 1099) 10.198727165699362
(281, 1102) 10.15076926747257
(281, 1106) 10.239553350822709
(281, 1107) 10.2205738693947

In [12]:
# print(depth.shape, np.max(depth), np.min(depth))
N = 0
test_depth = depth_map_sys
for i in range(375):
    for j in range(1242):
        if test_depth[i][j] != 0.0 and test_depth[i][j] <= 70 and j>1000:
            N+=1
            print((i,j), test_depth[i][j])
print("number of anchors", N)

(45, 1167) 10.802842
(46, 1164) 10.924005
(51, 1081) 13.344153
(52, 1076) 13.574025
(53, 1029) 15.175582
(56, 1095) 13.197295
(56, 1135) 11.791007
(58, 1014) 15.930045
(71, 1015) 16.000893
(71, 1017) 15.5736685
(78, 1017) 16.283504
(79, 1020) 15.442805
(79, 1053) 15.637683
(85, 1122) 12.579938
(88, 1158) 11.651252
(89, 1113) 13.045823
(89, 1114) 12.90627
(89, 1158) 11.464522
(93, 1071) 15.053643
(95, 1072) 14.694091
(99, 1020) 15.944016
(100, 1100) 13.095473
(101, 1097) 13.700807
(101, 1099) 13.719604
(105, 1133) 12.9190035
(113, 1099) 13.374267
(114, 1098) 13.275849
(114, 1100) 12.959149
(117, 1176) 11.659595
(126, 1001) 16.46962
(126, 1002) 16.39344
(126, 1003) 16.176052
(127, 1019) 15.694104
(135, 1042) 16.28746
(135, 1043) 15.821965
(136, 1041) 16.43192
(171, 1082) 25.665638
(176, 1009) 17.076313
(189, 1018) 16.052792
(190, 1017) 16.922062
(191, 1096) 25.92667
(191, 1097) 25.19069
(191, 1099) 24.507442
(192, 1098) 24.729713
(192, 1101) 24.204868
(194, 1055) 27.226162
(247, 1029) 13